In [1]:
%load_ext autoreload
%autoreload 2

In [52]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [5]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json

In [3]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v2.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.2, 0.5, 0.2, 0.1]
        )

In [4]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
    solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    return ac

In [107]:
d = {'extremelyPoor':[],
    'midRange':[],
    'gradeA':[]
    }
for f in list(goodFileDict.keys())[0:350]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        # try:
            # if result == None:
            #     print(f)
        # except (ZeroDivisionError, IndexError) as e:
        #     print("FROM HERE")
        #     result = None
        #     pass
        # except KeyError as e:
        #     print("KeyError:", e)
        #     print(f)


        if result != None:
            ac = Accuracy(result, dcsObj)
            if(ac < 20):
                d['extremelyPoor'].append(f)
            elif(ac > 30):
                if(ac < 50):
                    d['midRange'].append(f)
                elif(ac > 80):
                    d['gradeA'].append(f)
                    
            
print()
            
print(len(d['extremelyPoor']))
print(len(d['midRange']))
print(len(d['gradeA']))
print(d)


25
73
20
{'midRange': ['414970.p', '318639.p', '433875.p', '217229.p', '211085.p', '273610.p', '128515.p', '110599.p', '120991.p', '427125.p', '18461.p', '198471.p', '133458.p', '63385.p', '387288.p', '350810.p', '299664.p', '135962.p', '185945.p', '390136.p', '185693.p', '18178.p', '317986.p', '230374.p', '122594.p', '276850.p', '397641.p', '97843.p', '377975.p', '111782.p', '246259.p', '118016.p', '263310.p', '210831.p', '439887.p', '64156.p', '62826.p', '333391.p', '325101.p', '197518.p', '225227.p', '416414.p', '166175.p', '71156.p', '391562.p', '208048.p', '427669.p', '297589.p', '336520.p', '50716.p', '336935.p', '297312.p', '170401.p', '399016.p', '402885.p', '142515.p', '123469.p', '158412.p', '382023.p', '125971.p', '241672.p', '239291.p', '432601.p', '110697.p', '389374.p', '58342.p', '164457.p', '277773.p', '110939.p', '363444.p', '423440.p', '100493.p', '185401.p'], 'extremelyPoor': ['403169.p', '17535.p', '307007.p', '233280.p', '390187.p', '318977.p', '425999.p', '277647

In [108]:
d['midRange'] = d['midRange'][0:15]
d['extremelyPoor'] = d['extremelyPoor'][0:15]
d['gradeA'] = d['gradeA'][0:15]

In [109]:

with open('ratedFiles.json','w') as fp:
    json.dump(d, fp)

In [7]:
d = json.load(open('ratedFiles.json'))
d

{'extremelyPoor': ['403169.p',
  '17535.p',
  '307007.p',
  '233280.p',
  '390187.p',
  '318977.p',
  '425999.p',
  '277647.p',
  '104112.p',
  '122203.p',
  '72226.p',
  '52560.p',
  '436227.p',
  '153308.p',
  '314269.p'],
 'gradeA': ['191500.p',
  '343805.p',
  '103514.p',
  '310372.p',
  '60725.p',
  '6735.p',
  '58608.p',
  '398980.p',
  '261260.p',
  '336143.p',
  '221038.p',
  '2189.p',
  '167435.p',
  '228186.p',
  '160056.p'],
 'midRange': ['414970.p',
  '318639.p',
  '433875.p',
  '217229.p',
  '211085.p',
  '273610.p',
  '128515.p',
  '110599.p',
  '120991.p',
  '427125.p',
  '18461.p',
  '198471.p',
  '133458.p',
  '63385.p',
  '387288.p']}

In [39]:
d2 = {'extremelyPoor':[],
    'midRange':[],
    'gradeA':[]
    }
for f in d['gradeA']:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        try:
            result, stat = algo.predictVerbose(sentenceObj, dcsObj)
        except IndexError:
            pass
        # try:
            # if result == None:
            #     print(f)
        # except (ZeroDivisionError, IndexError) as e:
        #     print("FROM HERE")
        #     result = None
        #     pass
        # except KeyError as e:
        #     print("KeyError:", e)
        #     print(f)


        if result != None:
            ac = Accuracy(result, dcsObj)
            d2['gradeA'].append(stat)
                
for f in d['extremelyPoor']:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        try:
            result, stat = algo.predictVerbose(sentenceObj, dcsObj)
        except IndexError:
            pass
        # try:
            # if result == None:
            #     print(f)
        # except (ZeroDivisionError, IndexError) as e:
        #     print("FROM HERE")
        #     result = None
        #     pass
        # except KeyError as e:
        #     print("KeyError:", e)
        #     print(f)


        if result != None:
            ac = Accuracy(result, dcsObj)
            d2['extremelyPoor'].append(stat)
            
for f in d['midRange']:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        
        result, stat = algo.predictVerbose(sentenceObj, dcsObj)
        
        # try:
            # if result == None:
            #     print(f)
        # except (ZeroDivisionError, IndexError) as e:
        #     print("FROM HERE")
        #     result = None
        #     pass
        # except KeyError as e:
        #     print("KeyError:", e)
        #     print(f)


        if result != None:
            ac = Accuracy(result, dcsObj)
            d2['midRange'].append(stat)
print("GO")          

GO


In [40]:
d2['gradeA'][3]

{'0': {'removed': [],
  'winner': {'forms': [{'indeclinable': ['part.']}],
   'lemmas': ['na'],
   'names': 'na'}},
 '1': {'removed': [{'forms': [{'noun': ['voc. pl. m.']}],
    'lemmas': ['sambhu'],
    'names': 'sambhavas'}],
  'winner': {'forms': [{'noun': ['nom. sg. m.']}, {'noun': ['nom. pl. m.']}],
   'lemmas': ['sambhava', 'sambhu'],
   'names': 'sambhavas'}},
 '2': {'removed': [],
  'winner': {'forms': [{'verb': ['pr. [2] ac. sg. 3']}],
   'lemmas': ['as_1'],
   'names': 'asti'}},
 'DCSLemma': [['sarva'], ['sarvatra'], ['sambhava'], ['na', 'as']],
 'sentence': 'sarvasya sarvatra saMBavo nAsti   '}

In [124]:
with open('stat_15.json', 'w') as fp:
    json.dump(d2, fp)

In [24]:
for key, val in d2.items():
    print(key, len(d2[key]))

gradeA 15
midRange 15
extremelyPoor 15


In [41]:
SeeSentence(sentenceObj)

sImAvfkze nipatite kurURAM samitikzaye   
Analyzing  sImAvfkze
0 :  sImAs ['sImA'] [{'noun': ['acc. pl. f.', 'nom. pl. f.']}]
0 :  sIma ['sIman'] [{'compound': ['iic.']}]
0 :  sImA ['sImA', 'sIman'] [{'noun': ['nom. sg. f.']}, {'noun': ['nom. sg. m.', 'nom. sg. f.']}]
3 :  a ['a'] [{'compound': ['iic.']}]
4 :  vfkze ['vfkza'] [{'noun': ['loc. sg. m.']}]
Analyzing  nipatite
0 :  ni ['n'] []
2 :  patite ['patta', 'pat_1'] [{'noun': ['acc. du. f.', 'nom. du. f.', 'acc. du. n.', 'nom. du. n.', 'loc. sg. n.', 'loc. sg. m.']}, {'verb': ['pp.']}]
2 :  patite ['patta', 'pat_1'] [{'noun': ['voc. du. f.', 'voc. sg. f.', 'voc. du. n.']}, {'verb': ['pp.']}]
Analyzing  kurURAm
0 :  kurURAm ['kuru'] [{'noun': ['g. pl. m.']}]
Analyzing  samitikzaye
0 :  samiti ['samt'] [{'compound': ['iic.']}]
0 :  samiti ['samt'] [{'noun': ['loc. sg. f.']}]
0 :  sa ['tad'] [{'noun': ['nom. sg. m.']}]
0 :  sa ['sa_1'] [{'compound': ['iic.']}]
9 :  ye ['ya_2'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'loc. sg. n.', 'a